In [ ]:
import IPython
import numpy as np

The basic idea of the **Longstaff-Schwartz** algorithm, described in detail in **Longstaff and Schwartz (2001)** (and similar approaches like those reported in **Clement and Protter (2002)**), is to use least-squares regression on a finite set of functions as a proxy for condtional expectation estimates. 

First of all, the time axis has to be discretized—i.e., if the American option is alive within the time horizon $[0, T ]$, early exercise is only allowed at discrete times $0 < t_1 < t_2 < ... < t_J = T$. The American option is thus approximated by a Bermudan option. For a particular exercise date $t_k$, early exercise is performed if the payoff from immediate exercise exceeds the continuation value—i.e., the value of the (remaining) option if it is not exercised at $t_k$. This continuation value can be expressed as conditional expectation of the option payoff with respect to the risk-neutral pricing measure $Q$. 

- The initial step of the actual algorithm is to determine the cashflow vector $C_{t_J}$ at the last timestep $t_J$ . These cashflows are easy to get because the continuation values are then zero, or in other words, it is directly the payoff of a vanilla call-option in the terminal value of each simulation $i$:

\begin{equation}
\begin{array}{lll}
C_{i,t_J}& =& \max \left(S_{i,t_J} - K,0\right)\\
\end{array}
\end{equation}


- Second, we consider the spot prices at time-step $t_{J−1}$, and estimate the **exercise value**, selecting those for which has positive payoff, or 

\begin{equation} \max \left(S_{i,t_{J-1}}-K,0\right) > 0 \end{equation} 


- In order to obtain the mentioned **continuation values**, **Longstaff-Schwartz** regress the discounted future cash-flows realized from continuing onto a finite set of basis functions of our values for the spot price. The regression is done by using the values from all of the paths. The set of the basis functions for the regression, in this notebook, is a polynomial regression (of 5 degrees) but it could also be *Hermite, Legendre, Chebyshev, Gegenbauer, or Jacobi polynomials*, for example. If $S$ is the spot price, $a_j$ are coefficients and $B_j$ is the set of basis functions, then the **continuation value** for a path $i$ with values $S_{i,t_n}$ at time $t_{n}$ is

\begin{equation}
\begin{array}{lll}
Cont_{i,t_{n-1}}& =& \sum_{j=0}^{\infty} a_j\left(t_{n}\right)B_j(S)\\
\end{array}
\end{equation}



- Once we have the **continuation values** and **exercise values** , we will perform **early exercise condition of the american option** whenever

\begin{equation}
\begin{array}{lll}
C_{i,t_{n-1}}& >& Cont_{i,t_{n-1}}\\
\end{array}
\end{equation}


- And finally, we then **step backward** through time, until we reach the *first time-step*. At each time-step, early exercise is performed as described previously. Note that whenever a cashflow at timestep $t_k$ is generated by early exercise in path $i$, all cashflows that occur in this path later than $t_k$ (this is, at most, one) have to be removed.


- Once the whole backward process reach the initial point, we can build a cashflow or **value matrix** from the cashflow vectors $C_{i,t_{n}}$ by concatenating the cashflow vectors $C_{i,t_{n}}$, $n = 1,\ldots,J$, and the option value is given by the arithmetic average of the row sums.